In [1]:
#
# Individual.py
#
#

import math
import copy
from random import Random

#A simple 1-D Individual class
class Individual:
    """
    Individual
    """
    minSigma=1e-100
    maxSigma=1
    learningRate=1
    minLimit=None
    maxLimit=None
    uniprng=None
    normprng=None
    fitFunc=None
    problemType = None #Picks which problem it belongs to

    def __init__(self,
                 latticeLength = None,
                 numParticleType = None,
                 vectorLength    = None,
                 ):
        #Different initilzation
        if self.problemType == 'Problem2':
            self.vectorLength = vectorLength
            self.sequence = [self.uniprng.uniform(self.minLimit,self.maxLimit) for _ in range(self.vectorLength)]
        else:
            self.numParticleType = numParticleType
            self.latticeLength = latticeLength
            self.sequence = [self.uniprng.randrange(0,self.numParticleType) for _ in range(self.latticeLength)]

        self.fit= None
        self.sigma=self.uniprng.uniform(0.9,0.1) #use "normalized" sigma

    def crossover(self, other):
        child = copy.deepcopy(self)
        child.sequence = []
        sequenceLength = len(self.sequence)
        #Set alpha
        alpha= self.uniprng.uniform(0,1)
        #Portion of each sequence
        l1_length = int(sequenceLength*alpha)
        l2_length = sequenceLength - l1_length
        #Portion of self piece and other piece needed for crossOver
        #Piece them together Creating new child
        s1 = self.uniprng.sample(self.sequence,l1_length)
        s2 = self.uniprng.sample(other.sequence,l2_length)
        childSequence = s1 + s2
        child.sequence = childSequence

        return child

    def mutate(self):

        self.sigma=self.sigma*math.exp(self.learningRate*self.normprng.normalvariate(0,1))
        if self.sigma < self.minSigma: self.sigma=self.minSigma
        if self.sigma > self.maxSigma: self.sigma=self.maxSigma

        if self.sigma > 0.3:
            if self.problemType == 'Problem2':
                #Float Vector CrossOver
                for i,x in enumerate(self.sequence):
                    #Shift the value of x to postive first
                    tmp = x + abs(self.minLimit)
                    #Bias can only be positive, there would be no negative bias
                    bias = (self.maxLimit-self.minLimit)*self.sigma*self.normprng.normalvariate(0,1)
                    tmp = bias + tmp
                    #Shift x back
                    self.sequence[i] = tmp - abs(self.minLimit)
            else:
                #QuantumLattice CrossOver
                #Mutation by changing the quantum types within a lattice
                NumOfMutateParticles = self.uniprng.randrange(0,int(self.sigma*self.latticeLength)+1)
                for _ in range(NumOfMutateParticles):
                    self.uniprng.shuffle(self.sequence)
                    self.sequence.pop()
                    self.sequence.append(self.uniprng.randrange(0,self.numParticleType))

    def evaluateFitness(self,
                        interactionMatrix = None,
                        selfEnergyVector  = None):
        if self.problemType == 'Problem1':
            self.fit=self.__class__.fitFunc(self.sequence,interactionMatrix,selfEnergyVector)
        else:
            self.fit=self.__class__.fitFunc(self.sequence)

    def __str__(self):
        return "\n"
    
    def display_info(self):
        print("--------------------------------\n[INFO]\n")
        print(f"Sequence: \n {self.sequence}")
        print(f"Parameters: \n sigma = {self.sigma}")
        


# Initilization

## Test Qunatum Lattice Problem1

In [2]:
#Fit function for problem1
#Finding total energy of lattice
def CalulateEnergy(lattice,interactionMatrix,selfEnergyVector):
    totalEnergy = 0

    #Energy by self-energy
    for quantum in lattice:
        #print(f"Type extracted : {type(selfEnergyVector[quantum])} of value : {selfEnergyVector[quantum]}")
        totalEnergy += selfEnergyVector[quantum]

    #print(f"total energy after self-energy:{totalEnergy}")
    #Energy by interaction
    for quantum in lattice:
        for interact_quantum in lattice:
            totalEnergy += interactionMatrix[quantum][interact_quantum]

    #print(f"Energy after interation:{totalEnergy}")
    return totalEnergy

In [3]:
randomSeed = 1234
problemType = "Problem1"

minLimit = 5.12
maxLimit = -5.12

#Set Params
interactionMatrix = [[10,4,1],[4,10,5],[1,5,10]]
selfEnergyVector  = [1,2,3]

#start random number generators
uniprng=Random()
uniprng.seed(randomSeed)
normprng=Random()
normprng.seed(randomSeed+101)

Individual.minLimit=minLimit
Individual.maxLimit=maxLimit
Individual.fitFunc=None
Individual.uniprng=uniprng
Individual.normprng=normprng
Individual.problemType=problemType


In [4]:
individual1 = Individual(numParticleType=3,latticeLength=10)
individual2 = Individual(numParticleType=3,latticeLength=10)

individual1.display_info()
individual2.display_info()

--------------------------------
[INFO]

Sequence: 
 [1, 0, 0, 0, 2, 0, 2, 2, 0, 0]
Parameters: 
 sigma = 0.2868152537665629
--------------------------------
[INFO]

Sequence: 
 [0, 0, 0, 0, 1, 2, 2, 1, 2, 1]
Parameters: 
 sigma = 0.78115628903337


## CrossOver

In [5]:
child = individual1.crossover(individual2)

child.display_info()

--------------------------------
[INFO]

Sequence: 
 [0, 0, 2, 1, 0, 1, 1, 0, 2, 0]
Parameters: 
 sigma = 0.2868152537665629


## Mutation

In [6]:
child.mutate()
individual1.mutate()
individual2.mutate()

individual1.display_info()
individual2.display_info()
child.display_info()


--------------------------------
[INFO]

Sequence: 
 [1, 0, 0, 0, 2, 0, 2, 2, 0, 0]
Parameters: 
 sigma = 0.20340463233893086
--------------------------------
[INFO]

Sequence: 
 [1, 2, 1, 1, 1, 0, 0, 1, 2, 2]
Parameters: 
 sigma = 0.968006676656815
--------------------------------
[INFO]

Sequence: 
 [0, 0, 2, 1, 0, 1, 1, 0, 2, 0]
Parameters: 
 sigma = 0.37832277828962896


## Test Problem2

In [7]:
randomSeed = 1234
problemType = "Problem2"

minLimit = 5.12
maxLimit = -5.12

#Set Params
interactionMatrix = [[10,4,1],[4,10,5],[1,5,10]]
selfEnergyVector  = [1,2,3]

#start random number generators
uniprng=Random()
uniprng.seed(randomSeed)
normprng=Random()
normprng.seed(randomSeed+101)

Individual.minLimit=minLimit
Individual.maxLimit=maxLimit
Individual.fitFunc=None
Individual.uniprng=uniprng
Individual.normprng=normprng
Individual.problemType=problemType

In [8]:
individual1 = Individual(vectorLength=10)
individual2 = Individual(vectorLength=10)

individual1.display_info()
individual2.display_info()

--------------------------------
[INFO]

Sequence: 
 [-4.776484205487502, 0.6068981844443888, 5.043287346600067, -4.208393855479032, -4.4981145330049435, -0.8420103481236394, -1.7568100504369673, 4.26047255718826, -2.7287647517879945, 2.6950679002816744]
Parameters: 
 sigma = 0.8753487826187121
--------------------------------
[INFO]

Sequence: 
 [-2.9570326244995426, 1.5760489922854597, -1.2624023044010855, -1.1859527178610199, 3.5988004996271354, 3.2451517705223, 3.948411190387839, 4.9703036878140505, 0.1356642242072743, -4.7605919837820485]
Parameters: 
 sigma = 0.8483501752182512


In [9]:
child = individual1.crossover(individual2)

child.display_info()

--------------------------------
[INFO]

Sequence: 
 [4.26047255718826, 0.6068981844443888, -2.7287647517879945, -4.4981145330049435, -1.7568100504369673, -2.9570326244995426, 3.5988004996271354, 1.5760489922854597, 3.2451517705223, -1.2624023044010855]
Parameters: 
 sigma = 0.8753487826187121


In [10]:
child.mutate()
individual1.mutate()
individual2.mutate()

individual1.display_info()
individual2.display_info()
child.display_info()

--------------------------------
[INFO]

Sequence: 
 [-4.776484205487502, 0.6068981844443888, 5.043287346600067, -4.208393855479032, -4.4981145330049435, -0.8420103481236394, -1.7568100504369673, 4.26047255718826, -2.7287647517879945, 2.6950679002816744]
Parameters: 
 sigma = 0.22221752113062543
--------------------------------
[INFO]

Sequence: 
 [9.864567766472327, -25.676200048568102, -1.888019693809209, -5.384858343920737, 11.127510559375008, 20.640302802567774, -4.833895599770579, 2.482719917087989, -16.950985893102963, -2.7188965544643175]
Parameters: 
 sigma = 0.9659519790052071
--------------------------------
[INFO]

Sequence: 
 [7.779356760751374, -1.5892105214171317, -2.051362427415479, 11.617283856319432, 2.6149997117813157, 7.078064096915463, 10.249421813289544, -4.7833377997812185, 12.802729532875453, -6.437903804853022]
Parameters: 
 sigma = 1
